In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/our-data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install lazypredict
import lazypredict
from lazypredict.Supervised import LazyRegressor
from sklearn.linear_model import LogisticRegression,RidgeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,PowerTransformer
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, recall_score, precision_score,recall_score, f1_score,confusion_matrix
import sklearn

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def results(model,x_test,y_test):
    print("R^2:"+str((model.score(x_test,y_test)))+ " MSE:"+str(metrics.mean_squared_error(model.predict(x_test),y_test))+" MAE:"+str(np.sqrt(metrics.mean_squared_error(model.predict(x_test),y_test))))
    res=model.predict(x_test)-y_test
    plt.scatter(model.predict(x_test),y=y_test)
    return res

In [ ]:
def Train_and_test(model):
    model.fit(x_train,y_train)
    model(ridge,x_test,y_test)

In [ ]:
import pandas as pd
data = pd.read_csv('/kaggle/input/cleaned-cars-sentiment/cleaned_cars_with_sentiment.csv')
#we are interested only in US market 

In [ ]:
data=data.loc[data["lat"]>0]
data=data.loc[data["long"]<-50]

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer
#sample because its untrainable otherwise USE RAM AND BIGGER SAMPLE
subset_size = 4160
subset_df = data.sample(n=subset_size, random_state=42)
coordinates = subset_df[['lat', 'long']].values

n_clusters = 89 #cities above 250000
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
subset_df['cluster'] = kmeans.fit_predict(coordinates)
city=kmeans.predict(data[['lat','long']])
cluster_avg = subset_df.groupby('cluster')[['lat', 'long']].mean().reset_index()
data["place_cluster"]=city
plt.figure(figsize=(10, 6))
plt.scatter(cluster_avg['long'], cluster_avg['lat'], alpha=0.8)
plt.title('Average Values for Each Cluster')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

In [ ]:
from yellowbrick.cluster import KElbowVisualizer
visualizer = KElbowVisualizer(kmeans, k=[10,37,89,315,761,1502]) #https://www.statista.com/statistics/241695/number-of-us-cities-towns-villages-by-population-size/
visualizer.fit(coordinates)
visualizer.show()

In [ ]:
pd.set_option('display.max_columns', 50)
print(data.shape)
data.head()


In [ ]:
data_export=data.drop(["id","region","description","state","lat","long"],axis=1)

In [ ]:
data_export.to_csv("data_knime.csv", index=False)

In [ ]:
import os
os.remove("data_knime.csv")

In [ ]:
y_raw=data["price"]

In [ ]:
y_raw.to_csv("y_raw.csv", index=False)

In [ ]:
data_dummies=pd.get_dummies(data.drop(["price","id","region","description","state","lat","long"],axis=1))


In [ ]:
data_dummies=pd.get_dummies(data_dummies,columns=["place_cluster"])

In [ ]:
data_dummies.to_csv("data_dummies.csv",index=False)

# NN AREA

In [ ]:
import numpy as np

In [ ]:
def min_max_scaler(X):
    return (X-min(X))/(max(X)-min(X))

In [ ]:
data_dummies_scaled=data_dummies
for column in ["odometer","age","usage","neg","neu","pos","compound"]:
    data_dummies_scaled[column]=min_max_scaler(data_dummies_scaled[column])

In [ ]:
data_dummies_scaled

In [ ]:
y_scaled=min_max_scaler(y_raw)

In [ ]:
x_train,x_test,y_train,y_test=sklearn.model_selection.train_test_split(data_dummies_scaled,y_scaled, train_size=0.8, test_size=0.2, random_state=69)

In [ ]:
pca=sklearn.decomposition.PCA(n_components=300)

In [ ]:
pca.fit(data_dummies_scaled)

In [ ]:
pca.explained_variance_ratio_.sum() 

In [ ]:
from sklearn.neural_network import MLPRegressor

In [ ]:
x_pca=pca.transform(data_dummies_scaled)

In [ ]:
x_pca

In [ ]:
def learning_valid(model):
    from sklearn.model_selection import learning_curve
    train_sizes, train_scores, valid_scores=learning_curve(estimator=model,X=x_train,y=y_train)
    plt.plot(train_sizes,train_scores),plt.plot(train_sizes,valid_scores)

In [ ]:
nn=MLPRegressor(hidden_layer_sizes=(150,100,100),verbose=True, tol=1e-5,solver="adam")
x_train,x_test,y_train,y_test=sklearn.model_selection.train_test_split(x_pca,y_scaled, train_size=0.8, test_size=0.2, random_state=69)

In [ ]:
nn.fit(x_train,y_train)

In [ ]:
nn.score(x_test,y_test)

In [ ]:
results(nn,x_test,y_test)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(nn.loss_curve_)
plt.plot(nn.validation_scores_)

In [ ]:
plt.hexbin(nn.predict(x_test),y_test,cmap="Blues",)